In [4]:
import os
from constants import *

In [3]:
import pandas as pd
with open(os.path.join(ROOT_PATH, "data", "2021AB_SN", "SRDEF")) as f:
    df_types = pd.read_csv(f, delimiter='|', names=["Record Type", "TUI", "Name", "Tree Number", "Definition", "Examples", "Usage note", "non-human flag ?", "Abbreviation", "Inverse"], index_col=False)
    df_types.drop(["Examples", "Usage note", "non-human flag ?"], axis=1, inplace=True)
    sty_df = df_types.loc[df_types["Record Type"] == "STY"].reset_index(drop=True)
    rl_df =  df_types.loc[df_types["Record Type"] == "RL"].reset_index(drop=True)

In [6]:
with open(os.path.join(ROOT_PATH, "data", "2021AB_SN", "SG")) as f:
    df_group = pd.read_csv(f, delimiter='|', names=["Group Abbrev", "Group Name", "TUI", "Semantic Type"], index_col=False)
TUI_map = {}
for i, infos in df_group[["Group Abbrev", "TUI"]].iterrows():
    TUI_map[infos["TUI"]] = infos["Group Abbrev"]

In [287]:
entities_df

,Word,Type,Source,StartChar,EndChar,Document,CUI,Score,TUI,Group,StartWord,EndWord
0,investigate,ENTITY,SciSpacy MD,3,14,28259203,C1292732,1.0,T169,CONC,1,1
1,effect,ENTITY,SciSpacy MD,19,25,28259203,C1280500,1.0,T080,CONC,3,3
2,blood transfusion,ENTITY,SciSpacy MD,29,47,28259203,C0005841,1.0,T061,PROC,5,6
3,risk,ENTITY,SciSpacy MD,55,59,28259203,C0035647,1.0,T078,CONC,9,9
4,anastomotic leakage,ENTITY,SciSpacy MD,63,82,28259203,C0919691,0.9999998807907104,T046,DISO,11,12
...,...,...,...,...,...,...,...,...,...,...,...,...
941,pathogenesis,ENTITY,SciSpacy MD,18859,18871,28259203,C0543483,1.0,T169,CONC,2780,2780
942,research,ENTITY,SciSpacy MD,18887,18895,28259203,C0035168,1.0,T062,PROC,2783,2783
943,Danish Cancer Society,ENTITY,SciSpacy MD,18949,18970,28259203,UNDEF,UNDEF,UNDEF,UNDEF,2792,2794
944,Aage,ENTITY,SciSpacy MD,18976,18980,28259203,UNDEF,UNDEF,UNDEF,UNDEF,2796,2796


In [288]:
import os
import pandas as pd

from constants import *

MAX_WORD_DIST = 20

relations_dicts = []


def get_UMLS_score(StartTUI, EndTUI, umls_relations_df):
    return len(umls_relations_df["RelationTUI"].loc[umls_relations_df["FirstTUI"]
                   == StartTUI].loc[umls_relations_df["EndTUI"] == EndTUI])

with open(os.path.join(ENTITIES_PATH, filename.split(".")[0]+".csv"), "r") as f:
    entities_df = pd.read_csv(f).drop("Unnamed: 0", axis=1)

with open(os.path.join(ROOT_PATH, "data", "2021AB_SN", "SRSTRE1")) as f:
    umls_relations_df = pd.read_csv(f, delimiter='|', names=["FirstTUI", "RelationTUI", "EndTUI"], index_col=False)

relations_dicts = []

for i in tqdm(range(len(entities_df))):
    forward_df = entities_df.iloc[i + 1:]
    valid_relations = forward_df.loc[forward_df["EndWord"] <= entities_df["StartWord"].iloc[i] + MAX_DIST]
    for j in range(len(valid_relations)):
        relations_dicts.append({"First": i,
                                "End": i + j + 1,
                                "FirstWord": entities_df.iloc[i]["Word"],
                                "EndWord": valid_relations.iloc[j]["Word"],
                                "FirstTUI": entities_df.iloc[i]["TUI"],
                                "EndTUI": valid_relations.iloc[j]["TUI"],
                                "FirstGroup": entities_df.iloc[i]["Group"],
                                "EndGroup": valid_relations.iloc[j]["Group"],
                                "Distance": valid_relations.iloc[j]["StartWord"] - entities_df.iloc[i]["EndWord"],
                                "ScoreUMLS": get_UMLS_score(entities_df.iloc[i]["TUI"], entities_df.iloc[j]["TUI"], umls_relations_df)})

pd.DataFrame(relations_dicts)

  0%|          | 0/946 [00:00<?, ?it/s]

,First,End,FirstWord,EndWord,FirstTUI,EndTUI,FirstGroup,EndGroup,Distance,ScoreUMLS
0,0,1,investigate,effect,T169,T080,CONC,CONC,2,0
1,0,2,investigate,blood transfusion,T169,T061,CONC,PROC,4,0
2,0,3,investigate,risk,T169,T078,CONC,CONC,8,0
3,0,4,investigate,anastomotic leakage,T169,T046,CONC,DISO,10,1
4,0,5,investigate,patient,T169,T101,CONC,LIVB,13,0
...,...,...,...,...,...,...,...,...,...,...
31259,942,944,research,Aage,T062,UNDEF,PROC,UNDEF,13,0
31260,942,945,research,Johanne Louis-Hansens Foundation,T062,UNDEF,PROC,UNDEF,15,0
31261,943,944,Danish Cancer Society,Aage,UNDEF,UNDEF,UNDEF,UNDEF,2,0
31262,943,945,Danish Cancer Society,Johanne Louis-Hansens Foundation,UNDEF,UNDEF,UNDEF,UNDEF,4,0


In [9]:
def get_relations_score(StartTUI, EndTUI, distance, umls_relations_df):
    num_umls = len(umls_relations_df["RelationTUI"].loc[umls_relations_df["FirstTUI"] == StartTUI].loc[umls_relations_df["EndTUI"] == EndTUI])
    if num_umls == 0:
        return STANDARD_SCORE
    else:
        return num_umls * UMLS_SCORE * (1 - (DIST_MULT * (distance - 1) / MAX_DIST))

In [ ]:
import os

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from constants import *

THRESHOLD = 80

SCORE_UMLS_MULT = 3
STANDARD_SCORE = 1
DIST_CST = 40  # Value used in the exponential, increase to make the distance less impactful

mega_dict = {}

with open(os.path.join(ROOT_PATH, "data", "2021AB_SN", "SRSTRE1")) as f:
    umls_relations_df = pd.read_csv(f, delimiter='|', names=["FirstTUI", "RelationTUI", "EndTUI"], index_col=False)

def get_score(distance, score_umls): # Score UMLS : Number of Matching realtions found in database
    if score_umls == 0:
        return STANDARD_SCORE
    else:
        return score_umls * SCORE_UMLS_MULT * np.exp(-1 * distance / DIST_CST)


for filename in os.listdir(RELATIONS_PATH):
    with open(os.path.join(RELATIONS_PATH, filename), "r") as f:
        relations_df = pd.read_csv(f)

    if "FirstWord" in relations_df.columns and  "EndWord" in relations_df.columns and "Distance" in relations_df.columns and "ScoreUMLS" in relations_df.columns:
        for first_word, end_word, firstTUI, endTUI, distance, score in list(relations_df[["FirstWord", "EndWord", "FirstTUI", "EndTUI", "Distance", "ScoreUMLS"]].itertuples(index=False, name=None)):
            if first_word and first_word not in mega_dict.keys():
                mega_dict[first_word] = {}
            else:
                num_umls = len(umls_relations_df["RelationTUI"].loc[umls_relations_df["FirstTUI"] == firstTUI].loc[umls_relations_df["EndTUI"] == endTUI])
                if end_word and end_word not in mega_dict[first_word].keys():
                    mega_dict[first_word][end_word] = get_score(distance, score)
                else:
                    mega_dict[first_word][end_word] += get_score(distance, score)

In [ ]:
def one_step(k1):
    if k1 not in mega_dict.keys():
        return None
    else:
        print(k1)
        edges = []
        for k2 in mega_dict[k1].keys():
            if mega_dict[k1][k2] >= THRESHOLD:
                print(k2)
                edges += [(k1, k2), one_step(k2)]
        return edges

In [ ]:
K = list(mega_dict.keys())[1]
print(one_step(K))

In [2]:
import os

import spacy
import scispacy
import pandas as pd

from tqdm.auto import tqdm

from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector

from constants import *

In [26]:
nlp_scispacy = spacy.load("en_core_sci_md")
nlp_scispacy.add_pipe("abbreviation_detector")
nlp_scispacy.add_pipe("scispacy_linker", config={"resolve_abbreviations": True,
                      "linker_name": "umls", "max_entities_per_mention": 1})

/home/romainc/anaconda3/envs/Anastomo_NER/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/romainc/anaconda3/envs/Anastomo_NER/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [27]:
filename = "20978948.txt"
with open(os.path.join(DATA_CLEAN_PATH, filename), "r") as f:
    text = f.read()

In [28]:
doc = nlp_scispacy(text)
entities = list(doc.ents)

/home/romainc/anaconda3/envs/Anastomo_NER/lib/python3.10/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/home/romainc/anaconda3/envs/Anastomo_NER/lib/python3.10/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [29]:
for ent in entities:
    print(ent.lemma_)

colorectal
anastomotic leakage
complication
postoperative
morbidity
mortality
study
investigate
early detection
anastomotic leakage
clinical presentation
patient
rectal cancer
study
patient
elective low anterior
resection
peritoneal sample
abdominal drain
day
postoperatively
bacteriological study
cytokine measurement
patient
group
symptomatic
clinical evidence
anastomotic leakage
clinical evidence
AL
study
variable
hospital stay
wound infection
operative time
blood loss
height of anastomosis
intraperitoneal cytokine
microbiological study
peritoneal fluid
AL
patient
diagnose
postoperatively
median day
intraperitoneal
bacterial colonization
cytokine level
patient
clinical evidence
AL
wound infection
anastomotic leakage group
hospital stay
patient
anastomotic leakage
AL
group
operative time
blood loss
blood transfusion
height of the anastomosis
peritoneal
cytokine level
intraperitoneal
bacterial colonization
diagnostic tool
decision make
surgeon
early detection
anastomotic leak
colorectal

In [30]:
for abrv in doc._.abbreviations:
    print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

In [ ]:
for filename in tqdm(os.listdir(DATA_CLEAN_PATH)):
    with open(os.path.join(DATA_CLEAN_PATH, filename), "r") as f:
        text = f.read()
    doc = nlp_scispacy(text)
    for abrv in doc._.abbreviations:
        print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

  0%|          | 0/702 [00:00<?, ?it/s]

In [ ]:
print(doc._.abbreviations)

In [21]:
doc = nlp_scispacy("This is a dummy abbreviation test, also known as DAT")

/home/romainc/anaconda3/envs/Anastomo_NER/lib/python3.10/site-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


In [22]:
for abrv in doc._.abbreviations:
    print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

In [50]:
filename = "19912285_RE.csv"

with open(os.path.join(RELATIONS_PATH, filename), "r") as f:
    relations_df = pd.read_csv(f).drop("Unnamed: 0", axis=1)

with open(os.path.join(ENTITIES_PATH, filename.split("_")[0]+".csv"), "r") as f:
    entities_df = pd.read_csv(f).drop("Unnamed: 0", axis=1)

In [51]:
typed_relations_df = relations_df.join(entities_df["Type"], on=["First"], rsuffix="_first")
typed_relations_df = typed_relations_df.join(entities_df["Type"], on=["End"], rsuffix="_end")

In [53]:
relations_df = relations_df.iloc[typed_relations_df.loc[typed_relations_df["Type"] == "ENTITY"].loc[typed_relations_df["Type_end"] == "ENTITY"].index].reset_index(drop=True)

In [57]:
typed_relations_df = relations_df.join(entities_df["Type"], on=["First"], rsuffix="_first")
typed_relations_df = typed_relations_df.join(entities_df["Type"], on=["End"], rsuffix="_end")
relations_df = relations_df.iloc[typed_relations_df.loc[typed_relations_df["Type"] == "ENTITY"].loc[typed_relations_df["Type_end"] == "ENTITY"].index].reset_index(drop=True)

In [306]:
total_sum = 0
num_entries = 0
max_dist = 0
cnt_files = 0
shitty_quantile = 0
p = 0.8

for filename in os.listdir(RELATIONS_PATH):
    
    cnt_files += 1
    
    with open(os.path.join(RELATIONS_PATH, filename), "r") as f:
        relations_df = pd.read_csv(f).drop("Unnamed: 0", axis=1)
    
    max_dist = max(max_dist, relations_df["Distance"].max())
    total_sum += sum(relations_df["Distance"])
    num_entries += len(relations_df)
    shitty_quantile += relations_df["Distance"].quantile(p)

In [304]:
print(total_sum/num_entries)
print(max_dist)

49.774383215743555
101


In [307]:
print(shitty_quantile/cnt_files)

79.17261055634809


In [310]:
sorted(("jajajja", "ahis is sparta"))

['ahis is sparta', 'jajajja']